In [1]:
import pandas as pd
import numpy as np
import data_processing
import torch
from torch.utils.data import Dataset ,DataLoader


In [3]:
import torch_xla


ImportError: libpython3.10.so.1.0: cannot open shared object file: No such file or directory

In [2]:
!pip install torch~=2.1.0 torch_xla[tpu]~=2.1.0 torchvision -f https://storage.googleapis.com/libtpu-releases/index.html


Looking in links: https://storage.googleapis.com/libtpu-releases/index.html
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.0 MB/s eta 0:00:0000:0100:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_6

In [33]:
import math
max_len = 5000
d_model=5
pe = torch.zeros(5000, d_model)
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term[:-1])

In [24]:
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))


In [29]:
pe[:, 0::2] = torch.sin(position * div_term)

In [35]:
pe[:, 1::2] = torch.sin(position * div_term[:-1])

In [28]:
div_term.shape

torch.Size([3])

In [23]:
div_term.shape

torch.Size([1])

In [5]:
position.shape

torch.Size([5000, 1])

In [4]:
pe.shape

torch.Size([5000, 5])

In [3]:
data= pd.read_csv('NASDAQ_3y/stock_price.csv')
# data.columns = ['Date', 'Ticker', 'Open', 'High','Low', 'Close', 'Volume','sell','buy' ]

In [12]:
data.columns = ['Date', 'Ticker', 'Open', 'High','Low', 'Close', 'Volume','sell','buy' ]

In [5]:
data = data.groupby('Ticker')

In [9]:
group_day_list =[]
for group in data:
  group_day_list.append(len(group[1]))

In [13]:
from collections import Counter

group_day_counts = Counter(group_day_list)
print(group_day_counts)


Counter({755: 2136, 724: 15, 605: 10, 671: 10, 754: 8, 604: 7, 511: 7, 705: 7, 584: 7, 608: 6, 520: 6, 546: 6, 649: 6, 527: 6, 589: 6, 552: 6, 742: 5, 676: 5, 711: 5, 613: 5, 609: 5, 703: 5, 652: 5, 659: 4, 667: 4, 744: 4, 678: 4, 738: 4, 541: 4, 526: 4, 619: 4, 727: 4, 585: 4, 734: 4, 313: 4, 743: 3, 590: 3, 655: 3, 721: 3, 600: 3, 522: 3, 657: 3, 472: 3, 658: 3, 617: 3, 532: 3, 739: 3, 512: 3, 594: 3, 702: 3, 507: 3, 618: 3, 544: 3, 736: 3, 574: 3, 637: 3, 665: 3, 748: 3, 448: 3, 718: 3, 690: 3, 660: 3, 645: 3, 708: 3, 715: 3, 723: 3, 515: 3, 614: 3, 681: 3, 635: 3, 591: 3, 700: 3, 445: 3, 749: 3, 747: 3, 745: 3, 595: 3, 728: 3, 513: 3, 753: 2, 624: 2, 668: 2, 453: 2, 400: 2, 257: 2, 673: 2, 304: 2, 427: 2, 362: 2, 516: 2, 687: 2, 641: 2, 698: 2, 383: 2, 517: 2, 395: 2, 701: 2, 587: 2, 717: 2, 642: 2, 686: 2, 467: 2, 535: 2, 493: 2, 737: 2, 227: 2, 672: 2, 459: 2, 621: 2, 628: 2, 1: 2, 586: 2, 476: 2, 489: 2, 540: 2, 713: 2, 521: 2, 629: 2, 547: 2, 323: 2, 639: 2, 750: 2, 531: 2, 492

In [19]:
WINDOW_SIZE  = 90
PREDICT_SIZE = 30
SLIDING_SIZE =1
FEATURE_SIZE = 6
HIDDEN_SIZE =40
BATCH_SIZE =64
FULLLY_SIZE =128
FEATURE = [ 'Open', 'High','Low', 'Volume','sell','buy' ]
TARGET = 'Close'
TICKER_NUMBER =2743
TOTAL_DAY = 166
WINDOW_NUMBER =int(TOTAL_DAY - (WINDOW_SIZE+PREDICT_SIZE)/SLIDING_SIZE+1)


In [20]:


def min_max(sequences):
  results = sequences.copy()
  v_min =results.min()
  v_max =results.max()
  new_min =0
  new_max =1
  min_max=[]
  for index,sequence in enumerate(results):
    v_min =sequence.min()
    v_max =sequence.max()
    v_p = (sequence - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    min_max.append([v_min,v_max])
    results[index] = v_p
  return results, min_max

def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)

def preprocess_lstm(df):
  sequences = list()
  for group in df.groupby('Ticker'):
    sequences.append(group[1][FEATURE])
  sequences=np.array(sequences)

  ## min_max trading
  price , trade,sentiments =np.split(sequences,[4,5],axis=2)
  trade , min_max_trading =min_max(trade)
  price , min_max_list =min_max(price)
  combine =np.concatenate([price,trade,sentiments],axis =2)

  ## min_max another

  ## min_max another
  result_list= []
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(combine,[i,i+120],axis=1)
    result_list.append(b)
  result_array = np.array(result_list)
  train ,vaild = np.split(result_array,[46],axis=0)
  train_x,train_y=np.split(train.reshape(46*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  return train_x,train_y,vaild_x,vaild_y,min_max_list

In [21]:
train_x,train_y,vaild_x,vaild_y,min_max_list = preprocess_lstm(data)


In [16]:
class BaseDataset(Dataset):
    def __init__(self, x_data, y_data, target):
        self.x_data = torch.tensor(x_data,dtype =torch.float32)
        self.y_data = torch.tensor(y_data[:,:,FEATURE.index(target)],dtype = torch.float32)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.x_data.shape[0]

In [18]:


data= pd.read_csv('stockPrice/NASDAQ.csv')
data.columns = ['Date', 'Ticker', 'Open', 'High','Low', 'Close', 'Volume','sell','buy' ]



def min_max(sequences):
  results = sequences.copy()
  v_min =results.min()
  v_max =results.max()
  new_min =0
  new_max =1
  min_max=[]
  for index,sequence in enumerate(results):
    v_min =sequence.min()
    v_max =sequence.max()
    v_p = (sequence - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    min_max.append([v_min,v_max])
    results[index] = v_p
  return results, min_max

def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)


def make_split(sequences,WINDOW_NUMBER,PREDICT_SIZE,WINDOW_SIZE,FEATURE_SIZE,TICKER_NUMBER,TOTAL_DAY):
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(sequences,[i,i+120],axis=1)
    original_list.append(b)
  result_array = np.array(result_list)
  train ,vaild = np.split(result_array,[46],axis=0)
  train_x,train_y=np.split(train.reshape((TOTAL_DAY-1)*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)


def preprocess_lstm(df):
  sequences = list()
  for group in df.groupby('Ticker'):
    sequences.append(group[1][FEATURE])
  sequences=np.array(sequences)

  ## min_max trading
  price , trade,sentiments =np.split(sequences,[4,5],axis=2)
  trade , min_max_trading =min_max(trade)
  price , min_max_list =min_max(price)
  combine =np.concatenate([price,trade,sentiments],axis =2)

  ## min_max another
  original_list= []
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(sequences,[i,i+120],axis=1)
    original_list.append(b)
  result_array = np.array(result_list)
  train ,vaild = np.split(result_array,[46],axis=0)
  train_x,train_y=np.split(train.reshape(46*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)

  ## min_max another
  result_list= []
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(combine,[i,i+120],axis=1)
    result_list.append(b)
  result_array = np.array(result_list)
  train ,vaild = np.split(result_array,[46],axis=0)

  train_x,train_y=np.split(train.reshape(46*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  return train_x,train_y,vaild_x,vaild_y,min_max_list

train_x,train_y,vaild_x,vaild_y,min_max_list = preprocess_lstm(data)
train_x.shape,train_y.shape,vaild_x.shape,vaild_y.shape,len(min_max_list)


class BaseDataset(Dataset):
    def __init__(self, x_data, y_data, target):
        self.x_data = torch.tensor(x_data,dtype =torch.float32)
        self.y_data = torch.tensor(y_data[:,:,FEATURE.index(target)],dtype = torch.float32)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.x_data.shape[0]

train_data = BaseDataset(train_x,train_y,TARGET)
test_data =  BaseDataset(vaild_x,vaild_y,TARGET)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)

ValueError: cannot reshape array of size 105989520 into shape (126178,120,6)

In [ ]:
train_data = BaseDataset(train_x,train_y,TARGET)
test_data =  BaseDataset(vaild_x,vaild_y,TARGET)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)

NameError: name 'BaseDataset' is not defined

In [ ]:
orginal5 = torch.reshape(train_data.x_data[0],(,90,6))

In [2]:
Window_maker.inverse_min_max()

NameError: name 'Window_maker' is not defined

In [4]:
import torch
torch.tensor(original[0])

NameError: name 'original' is not defined

train_x,train_y,vaild_x,vaild_y,min_max_list = preprocess_lstm(data)


In [ ]:


def min_max(sequences):
  results = sequences.copy()
  v_min =results.min()
  v_max =results.max()
  new_min =0
  new_max =1
  min_max=[]
  for index,sequence in enumerate(results):
    v_min =sequence.min()
    v_max =sequence.max()
    v_p = (sequence - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    min_max.append([v_min,v_max])
    results[index] = v_p
  return results, min_max

def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)

def preprocess_lstm(df):
  sequences = list()
  for group in df.groupby('Ticker'):
    sequences.append(group[1][FEATURE])
  sequences=np.array(sequences)

  ## min_max trading
  price , trade,sentiments =np.split(sequences,[4,5],axis=2)
  trade , min_max_trading =min_max(trade)
  price , min_max_list =min_max(price)
  combine =np.concatenate([price,trade,sentiments],axis =2)

  ## min_max another

  ## min_max another
  result_list= []
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(combine,[i,i+120],axis=1)
    result_list.append(b)
  result_array = np.array(result_list)
  train ,vaild = np.split(result_array,[46],axis=0)
  train_x,train_y=np.split(train.reshape(46*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  return train_x,train_y,vaild_x,vaild_y,min_max_list